In [4]:
from flask import Flask, request, jsonify
from tensorflow.keras.models import load_model
from gensim.models import Word2Vec
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from collections import Counter
import tensorflow as tf
from tensorflow.keras.initializers import Orthogonal


In [5]:

app = Flask(__name__)

nltk.download('stopwords')
stopwords_set = set(stopwords.words("english"))

def custom_lstm(**kwargs):
    kwargs.pop('time_major', None)
    return tf.keras.layers.LSTM(**kwargs)

model = load_model('modelo_lstmMeans.h5', custom_objects={'Orthogonal': Orthogonal, 'LSTM': custom_lstm})
w2v_model = Word2Vec.load("word2vecSentiments.model")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\perez\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\perez\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [6]:
from flask import Flask, request, jsonify
from tensorflow.keras.models import load_model
from gensim.models import Word2Vec
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from collections import Counter
import tensorflow as tf
from tensorflow.keras.initializers import Orthogonal

app = Flask(__name__)

nltk.download('stopwords')
stopwords_set = set(stopwords.words("english"))

def custom_lstm(**kwargs):
    kwargs.pop('time_major', None)
    return tf.keras.layers.LSTM(**kwargs)

model = load_model('modelo_lstmMeans.h5', custom_objects={'Orthogonal': Orthogonal, 'LSTM': custom_lstm})
w2v_model = Word2Vec.load("word2vecSentiments.model")

def limpiar_texto(texto):
    words_filtered = [e.lower() for e in texto.split() if len(e) >= 3]
    words_cleaned = [word for word in words_filtered if 'http' not in word and not word.startswith('@') and not word.startswith('#') and word != 'rt']
    words_without_stopwords = [word for word in words_cleaned if not word in stopwords_set]
    words_final = [re.sub(r'[^a-zA-Z]', '', word) for word in words_without_stopwords]
    return words_final

def media_embedding(opinion):
    embeddings = [w2v_model.wv[palabra] for palabra in opinion if palabra in w2v_model.wv.key_to_index]
    return np.mean(embeddings, axis=0) if embeddings else np.zeros(w2v_model.vector_size)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    array_strings = data.get('array_strings')

    if not array_strings:
        return jsonify({'error': 'No se proporcionaron frases para predecir.'}), 400

    predicciones = []
    for frase in array_strings:
        frase_limpia = limpiar_texto(frase)
        embedding = media_embedding(frase_limpia)

        # Redimensionar el embedding para que sea compatible con el modelo
        embedding = np.expand_dims(embedding, axis=0)  # Añadir dimensión batch
        embedding = np.expand_dims(embedding, axis=1)  # Añadir dimensión timesteps

        # Predecir la etiqueta
        prediccion = model.predict(embedding)

        # Convertir la predicción a etiqueta
        etiqueta = np.argmax(prediccion, axis=1)[0]
        predicciones.append(etiqueta)

    # Determinar la predicción más frecuente
    prediccionReal = Counter(predicciones).most_common(1)[0][0]

    return jsonify({'prediccionReal': int(prediccionReal)})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True, use_reloader=False)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\perez\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\perez\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.20.10.13:5000
INFO:werkzeug:Press CTRL+C to quit


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 532ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


INFO:werkzeug:172.20.10.11 - - [26/Jun/2024 13:39:22] "POST /predict HTTP/1.1" 200 -
